In [1]:
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import scipy.optimize 
from strategy1 import strategy1

In [45]:
factors = pd.read_csv("../../Factors/merge_Factors.csv")
factor_returns = pd.read_csv("../../Analysis/factors_returns_multi.csv")


In [46]:
date = "2022-10-03"
selected_stocks = strategy1("2022-10-03")
factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
factor_returns = factor_returns.loc[factor_returns["Date"]==date]
factor_returns = factor_returns.iloc[:,1:]
stock_list = factors["Index"].tolist()
factors = factors.iloc[:, 3:]
factors = np.array(factors)
factor_returns = np.array(factor_returns)

In [47]:
stock_return = np.dot(factors, factor_returns.T).reshape(-1)
stock_list = np.array(stock_list)

array([0.01065462, 0.02895356, 0.04407649, 0.03290185, 0.00890382,
       0.01174139, 0.04829632, 0.04372723, 0.01981285, 0.07887703])

In [46]:
from datetime import datetime 


def optimize(weights,tickers,start,end,calc_window,rf_rate,short=False,
             price_type='Close',clean='True'):
          
        '''
        weights; *type = list, initial guess weights
        
        start; input as str, data start date format 'YYYY-MM-DD' 
        end; input as str, data end date format 'YYYY-MM-DD'
        
        calc_window; input as str, calculation window for returns, 
        
        rf_rate; desired risk-free rate to benchmark against
        
        price_type; *input as str, default = 'Close', options = 'Adj Close', 
        'Close', 'High','Low','Open', 'Volume'
        
        short = bool, default = 'False', if true weight bounds -1 to 1
        
        clean; bool, default = 'False', if true, the function will drop NA rows
        '''
        start = datetime.strptime(start,'%Y-%m-%d')
        end = datetime.strptime(end,'%Y-%m-%d')
        
        prices = pdr.get_data_yahoo(tickers, start, end)[price_type]
        
        op = prices.pct_change(periods=int(calc_window))
        
        if clean == False:
            op
            
        else:
            op = op.dropna()
        
        vcv_matrix = np.cov(op.T)
        
        avg_return = op.mean()
        
        def sharpe(weights,vcv_matrix,avg_return):        
            
            var = np.dot(np.dot(weights,vcv_matrix),weights)
            
            ret =  np.dot(weights,avg_return)
            
            s_ratio = -(ret-rf_rate)/np.sqrt(var)
            
            return s_ratio
           
        
        sharpe_cons = ({'type':'eq','fun': lambda x: sum(x)-1})
        
        
        if short == False:
            sharpe_bnds = ((0,1),)*len(tickers)
        
        if short == True:
            sharpe_bnds = ((-1,1),)*len(tickers)
        
        optimal = scipy.optimize.minimize(sharpe,weights,bounds = sharpe_bnds,
                                 args = (vcv_matrix,avg_return),
                                 constraints = sharpe_cons)
        
        return optimal

In [27]:
d=pd.read_csv('trading_indicators.csv',index_col=0)

In [39]:
tickers=list(d['index'][0:10].values)

In [35]:
start='2022-08-01'
end='2022-10-31'

In [36]:
#不变，始终为1
calc_window='1'

In [42]:
rf_rate=0.02

In [48]:
a=optimize(weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')

In [50]:
a.x

array([-0.21949472,  0.06524437, -0.43674423,  1.        , -0.30397606,
       -0.09550615, -0.33532088,  1.        ,  0.00699695,  0.31880071])